Requirements
  1. numpy
  2. multiprocessing
  3. cython or cpython 
  4. pycuda
Introduction
  1. Nodes
  2. Processors/Sockets
  3. Cores
  4. Threads
  5. Logical processors



# 1. Multiprocessing

In [ ]:
# lscpu (Linux)
# sysctl -n hw.ncpu (mac)(Not tested)
# wmic cpu get caption, name, maxclockspeed, currentclockspeed, numberofcores, numberoflogicalprocessors (Windows)

## <span style="color:#44FF44;">Using multiprocessing.pool</span>
1. pool.map

In [ ]:
! python 00_tuto.py 1
! python 00_tuto.py 0


2. Job numbers and reducing execution time

In [ ]:
# ! python 01_tuto.py 1
! python 01_tuto.py 10
! python 01_tuto.py 14
! python 01_tuto.py 20

3. Using `pool.starmap`
   
For more than 1 argument

In [ ]:
! python 02_tuto.py 10

4. Using `pool.apply_async`

More control over the multiprocessing tasks <br>
Assigning multiple functions in a session <br>
Releassing the parent process for other tasksto do <br>

In [ ]:
! python 03_tuto.py

## <span style="color:#FFAA00;">Using multiprocessing.Process</span>
Let's leave this for children to play with and focus more on cool stuff<br>
Also, multiprocessing.Process is friendly with notebooks but... 


In [30]:
import multiprocessing

# Define a function to be executed by the process
def worker_function(name):
    print(f"Hello, {name}!")

if __name__=='__main__':
  # Create a multiprocessing.Process instance
  process1 = multiprocessing.Process(target=worker_function, args=("Charlie the Harley"  ,))
  process2 = multiprocessing.Process(target=worker_function, args=("Sophie the trophy"   ,))
  process3 = multiprocessing.Process(target=worker_function, args=("Albert the shepherd" ,))
  
  # Start the process
  process1.start()
  process2.start()
  process3.start()

  # Wait for the process to finish (optional)
  print("Main program continues...")

  process1.join()
  process2.join()
  process3.join()

Main program continues...


In [32]:
! python 04_tuto.py

Hello, Charlie the Harley!
Hello, Sophie the trophy!
Hello, Albert the shepherd!
Main program continues...


In the notebook: 
1. The kernel will kill itself (suicide) if any cell ended running and a process is alive. join all processes before ending a cell
2. If `Process` is called directly inside the notebook, the children can't reach the casting interface (can't `print()`) but it's possible if the Process is inside a module/script that have been imported by the notebook.
3. In any python script, `Process` and `Pool` should be called in main scope (`if __name__=='__main__':...`) because the script is called inside all processes.
4. We can use args or kwargs arguments in `Process` to pass arguments or keyworded arguments.
5. There is no number of processes to specify in this case so it should be manually done 

In [47]:
! python 05_tuto.py

PID  22140
PID  36440
PID  16008
PID  34856
PID  32824
PID  37412
PID  19736
PID  11996
PID  30428
PID  42324
PID  33520
PID  6280


Traceback (most recent call last):
  File "c:\Users\smzergua\Documents\paris saclay\07_Formations\Code Club\05_tuto.py", line 28, in <module>
    processes.pop(p)
TypeError: 'Process' object cannot be interpreted as an integer


`for ind,p in enumerate(processes): `<br>  
&emsp;`if not p.is_alive():`<br>
&emsp;&emsp;`p.close()`<br>
&emsp;&emsp;`processes.pop(p)`<br>

It's to close all the processes that have ended their jobs. This is important if you have a lot of processes >100s. There is some files created and opened and all systems have a limited amount of memory for opening files simultaneously.

## <span style="color:#FF5500;">Sharing  memory</span>